# AG2: Orchestrating agents (Nested chat)

- Orchestrating agents<br>
  - https://docs.ag2.ai/docs/user-guide/basic-concepts/orchestration/orchestrations
- Nested chat<br>
  - https://docs.ag2.ai/docs/user-guide/basic-concepts/orchestration/nested-chat

## Orchestrating agents

- **Two-agent chat**: The simplest form of conversation pattern where two agents chat back-and-forth with each other. This has been demonstrated in the previous examples.
- **Sequential chat**: A sequence of chats, each between two agents, chained together by a carryover mechanism (which brings the summary of the previous chat to the context of the next chat). Useful for simple sequential workflows.
- **Group chat**: A chat with more than two agents with options on how agents are selected. See GroupChat Overview for further details.
- **Nested chat**: A mechanism to package a workflow into a single agent/chat for reuse in a workflow.
- **Swarm**: A pattern based on agents with handoffs. There’s a shared context and each agent has tools and the ability to transfer control to other agents. The original swarm concept was created by OpenAI.

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## Nested chat

In [4]:
# from autogen import ConversableAgent, GroupChatManager, GroupChat, LLMConfig
from autogen import ConversableAgent, GroupChatManager, GroupChat

# llm_config = LLMConfig(
#     api_type="openai",
#     model="gpt-4o-mini",
# )
llm_config = {"api_type": "openai", "model": "gpt-4o-mini"}

# Curriculum Standards Agent
curriculum_agent = ConversableAgent(
    name="Curriculum_Agent",
    system_message="""You are a curriculum standards expert for fourth grade education.
    When given a topic, you provide relevant grade-level standards and learning objectives.
    Format every response as:
    STANDARDS:
    - [Standard 1]
    - [Standard 2]
    OBJECTIVES:
    - By the end of this lesson, students will be able to [objective 1]
    - By the end of this lesson, students will be able to [objective 2]""",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# Lesson Planner Agent
lesson_planner_agent = ConversableAgent(
    name="Lesson_Planner_Agent",
    system_message="""You are a lesson planning specialist.
    Given standards and objectives, you create detailed lesson plans including:
    - Opening/Hook (5-10 minutes)
    - Main Activity (20-30 minutes)
    - Practice Activity (15-20 minutes)
    - Assessment/Closure (5-10 minutes)
    Format as a structured lesson plan with clear timing and materials needed.""",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# Lesson Reviewer Agent
lesson_reviewer_agent = ConversableAgent(
    name="Lesson_Reviewer_Agent",
    system_message="""You are a lesson plan reviewer who ensures:
    1. Age-appropriate content and activities
    2. Alignment with provided standards
    3. Realistic timing
    4. Clear instructions
    5. Differentiation opportunities
    Provide specific feedback in these areas and suggest improvements if needed.""",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# Lead Teacher Agent
lead_teacher_agent = ConversableAgent(
    name="Lead_Teacher_Agent",
    system_message="""You are an experienced fourth grade teacher who oversees the lesson planning process.
    Your role is to:
    1. Initiate the planning process with a clear topic
    2. Review and integrate feedback from other agents
    3. Ensure the final lesson plan is practical and engaging
    4. Make final adjustments based on classroom experience""",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# Create the group chat for collaborative lesson planning
planning_chat = GroupChat(
    agents=[curriculum_agent, lesson_planner_agent, lesson_reviewer_agent],
    messages=[],
    max_round=4,
    send_introductions=True,
)

planning_manager = GroupChatManager(
    groupchat=planning_chat,
    llm_config=llm_config,
)

# Formatter of the final lesson plan to a standard format
formatter_message = """You are a lesson plan formatter. Format the complete plan as follows:
<title>Lesson plan title</title>
<standards>Standards covered</standards>
<learning_objectives>Key learning objectives</learning_objectives>
<materials>Materials required</materials>
<activities>Lesson plan activities</activities>
<assessment>Assessment details</assessment>
"""

lesson_formatter = ConversableAgent(
    name="formatter_agent",
    llm_config=llm_config,
    system_message=formatter_message,
)

# Create nested chats configuration
nested_chats = [
    {
        # The first internal chat determines the standards and objectives
        # A round of revisions is supported with max_turns = 2
        "recipient": curriculum_agent,
        "message": lambda recipient, messages, sender, config: f"Please provide fourth grade standards and objectives for the topic: {messages[-1]['content']}",
        "max_turns": 2,
        "summary_method": "last_msg",
    },
    {
        # A group chat follows, where the lesson plan is created
        "recipient": planning_manager,
        "message": "Based on these standards and objectives, create a detailed lesson plan.",
        "max_turns": 1,
        "summary_method": "last_msg",
    },
    {
        # Finally, a two-agent chat formats the lesson plan
        # The result of this will be the lead_teacher_agent's reply
        "recipient": lesson_formatter,
        "message": "Format the lesson plan.",
        "max_turns": 1,
        "summary_method": "last_msg",
    }
]

# Register nested chats with the lead teacher
lead_teacher_agent.register_nested_chats(
    chat_queue=nested_chats,
    trigger=lambda sender: sender not in [curriculum_agent, planning_manager, lesson_reviewer_agent],
)

# A human-in-the-loop agent
human = ConversableAgent(
    name="human_agent",
    human_input_mode="ALWAYS"
)

In [5]:
# A two-agent chat between our human and the lead_teacher_agent
# to demonstrate the full workflow is within the one agent
result = lead_teacher_agent.initiate_chat(
    recipient=human,
    message="What topic would you like to get a lesson plan for?",
    max_turns=2
)

Lead_Teacher_Agent (to human_agent):

What topic would you like to get a lesson plan for?

--------------------------------------------------------------------------------


Replying as human_agent. Provide feedback to Lead_Teacher_Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  AI Agents Development


human_agent (to Lead_Teacher_Agent):

AI Agents Development

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Lead_Teacher_Agent (to Curriculum_Agent):

Please provide fourth grade standards and objectives for the topic: AI Agents Development

--------------------------------------------------------------------------------
Curriculum_Agent (to Lead_Teacher_Agent):

STANDARDS:
- ISTE Standards for Students 4.  a. Empowered Learner: Students leverage technology to take an active role in choosing, achieving, and demonstrating competency in their learning goals.
- CSTA K-12 Computer Science Framework: 2-AP-10. Describe and apply appropriate tools and techniques to debug code.

OBJECTIVES:
- By the end of this lesson, students will be able to explain the basic functions of A

Replying as human_agent. Provide feedback to Lead_Teacher_Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


In [6]:
print("Final Lesson Plan:\n", result.summary)

Final Lesson Plan:
 <title>Introduction to AI Agents for Fourth Graders</title>
<standards>Common Core Standards for Mathematics and Technology Integration; ISTE Standards for Students</standards>
<learning_objectives>Students will understand the concept of AI agents and algorithms, recognize real-world applications of AI, create an algorithm flowchart, and prototype their AI agents using a basic coding platform.</learning_objectives>
<materials>
- Computers or tablets with internet access
- Whiteboard and markers
- Chart paper and markers for group presentations
- Access to coding platforms (Code.org, Scratch, or Tynker)
- Reflective journals or notebooks
- Guest speaker (optional)
</materials>
<activities>
1. **Introduction to AI Agents (20 minutes)**
   - Begin with a discussion on what AI agents are. Use familiar examples like Siri, Alexa, and educational games.
   - Show a short video or presentation about how AI is used in everyday life to engage students.

2. **Group Discussion 